In [1]:
import pandas as pd

In [6]:
# Loading rating dataframe
rating_df = pd.read_csv("input_data\\rating.csv")


In [12]:
# Loading movie dataframe
movie_df = pd.read_csv('input_data\\movie.csv')

# Use str.extract to separate the movie title and year from the 'title' column
movie_df[['title', 'year']] = movie_df['title'].str.extract(r'^(.*)\s\((\d{4})\)$')

# Perform one-hot encoding on the 'genres' column, creating binary columns for each genre
df_genres_encoded = movie_df['genres'].str.get_dummies(sep='|')

# Concatenate the one-hot encoded genres with the original DataFrame
movie_df = pd.concat([movie_df, df_genres_encoded], axis=1)

# Drop the 'genres', 'year', and '(no genres listed)' columns from the DataFrame
movie_df = movie_df.drop(columns=['(no genres listed)'], axis=1)


# FILTER FUNCTION CREATE RATING AND MOVIE DATAFRAMES

In [13]:
import pandas as pd

def filter_and_save_dataframes(rating_df, movie_df, col_name, lower_bound, upper_bound, output_filename):
    """
    Filters the DataFrame based on the number of ratings and writes the result to two CSV files.
    
    Parameters:
    df (DataFrame): The input DataFrame to be filtered.
    movie_df (DataFrame): The DataFrame containing movie details.
    col_name (str): The name of the column to count ratings per userId or movieId
    lower_bound (int): The lower bound for the number of ratings to filter.
    upper_bound (int): The upper bound for the number of ratings to filter.
    output_filename (str): The base name of the output CSV files (without '.csv').
    
    Returns:
    DataFrame: The filtered DataFrame.
    """
    # Calculate the number of ratings each movie or user has received
    rating_df[f'ratings_per_{col_name}'] = rating_df.groupby(col_name)['userId'].transform('count')
    
    # Filter the DataFrame
    filtered_df = rating_df[(rating_df[f'ratings_per_{col_name}'] > lower_bound) & 
                     (rating_df[f'ratings_per_{col_name}'] < upper_bound)]
    
    # Drop all columns except 'userId', 'movieId', 'rating'
    filtered_df = filtered_df[['userId', 'movieId', 'rating']]
    
    print(output_filename)
    # Print the number and percentage of ratings in the filtered DataFrame
    print("nb of ratings :",len(filtered_df))
    print("percentage of ratings :", round(len(filtered_df)/len(rating_df)*100),"%")

    # Print the number and percentage of unique users in the filtered DataFrame
    print("nb of users :",filtered_df['userId'].nunique())
    print("percentage of users :", round(filtered_df['userId'].nunique()/rating_df['userId'].nunique()*100),"%")

    # Print the number and percentage of unique movies in the filtered DataFrame
    print("nb of movies :",filtered_df['movieId'].nunique())
    print("percentage of movies :", round(filtered_df['movieId'].nunique()/rating_df['movieId'].nunique()*100),"%")
    
    # Write the filtered ratings DataFrame into a CSV
    filtered_df.to_csv(f'output_data/rating_{output_filename}.csv', index=False)
    
    # Dynamically set the name of the DataFrame
    globals()[output_filename] = filtered_df
    
    # Filter movie_df to keep only the movieIds present in filtered_df
    filtered_movie_df = movie_df[movie_df['movieId'].isin(filtered_df['movieId'])]
    
    # Check if the number of unique movieIds matches
    if len(filtered_movie_df) == filtered_df['movieId'].nunique():
        print("Success: The lengths match.")
    else:
        print("Error: Lengths do not match.")
    
    # Write the filtered movie DataFrame into a CSV
    filtered_movie_df.to_csv(f'output_data/movie_{output_filename}.csv', index=False)
    
    
    # Return the filtered DataFrame for further use if needed
    return filtered_df, filtered_movie_df


# Call the combined function
filter_and_save_dataframes(rating_df, movie_df, 'movieId', 3500, 70000, 'blockbusters')


blockbusters
nb of ratings : 14425247
percentage of ratings : 72 %
nb of users : 138489
percentage of users : 100 %
nb of movies : 1377
percentage of movies : 5 %
Success: The lengths match.


(          userId  movieId  rating
 0              1        2     3.5
 1              1       29     3.5
 2              1       32     3.5
 3              1       47     3.5
 4              1       50     3.5
 ...          ...      ...     ...
 20000249  138493    59315     4.0
 20000251  138493    59784     5.0
 20000252  138493    60069     4.0
 20000258  138493    68954     4.5
 20000261  138493    70286     5.0
 
 [14425247 rows x 3 columns],
        movieId                                         title  \
 0            1                                     Toy Story   
 1            2                                       Jumanji   
 2            3                              Grumpier Old Men   
 4            5                   Father of the Bride Part II   
 5            6                                          Heat   
 ...        ...                                           ...   
 17499    88125  Harry Potter and the Deathly Hallows: Part 2   
 17874    89745             